# llama3

In [1]:
! pip3 install transformers>=4.32.0 optimum>=1.12.0
! pip3 install auto-gptq

In [1]:
# model type
model_id = "TechxGenus/Meta-Llama-3-70B-Instruct-GPTQ"
model_size = "70B"
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_size = "8B"

In [2]:
# Custom parameters
column_type = "multi"                             # single/multi, like for single column type, We only read tables with only one column
directory_indexs = ["K0", "K1", "K2", "K3", "K4"] # directory used for iterate
file_num = 100                                    # for each directory, read file_num of column_type files
rows_num = 20                                     # for each table, how many rows we need read for prompt
max_new_tokens = 4096                             # max token for LLM
#ignore_mismatch = False                           # if throw out mismatch
gpu_device = "auto"                               # 8B needs specify the index of GPU like 1 or 2, 70B use "auto"

In [3]:
# import transformers
# import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map=gpu_device,
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_id,
#     # model_kwargs={"torch_dtype": torch.bfloat16},
#     device=gpu_device,
# )


/u/s/d/sdam/miniconda3/envs/llama2/lib/python3.12/site-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class


In [4]:
messages = [
                {"role": "system", "content": """
            You are a database expert who can make general predictions for missing column values in database tables, and the predicted column names are within the required candidate set. All output must be in valid JSON. Don't add explanation beyond the JSON.
            Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
            """},
                {"role": "user", "content": """
Column Names are limited to the following:
name, description, team, type, age, location, year, city, rank, status, state, category,
weight, code, club, artist, result, position, country, notes, class, company, album, symbol,
address, duration, format, county, day, gender, industry, language, sex, product, jockey,
region, area, service, teamName, order, isbn, fileSize, grades, publisher, plays, origin,
elevation, affiliation, component, owner, genre,  manufacturer, brand, family, credit, depth,
classification, collection, species, command, nationality, currency, range, affiliate,
birthDate, ranking, capacity, birthPlace, person, creator, operator, religion, education,
requirement, director, sales, continent, organisation
Do not use any column names aside from these.

Output must be in valid JSON like the following example {"colnames" : ["col1", "col2"]}

Given the following relational table:

IDI1,isopentenyl-diphosphate delta isomerase..isopentenyl-diphosphate delta isomerase.
FDFT1,farnesyl-diphosphate farnesyltransferase 1.DGPT.farnesyl-diphosphate farnesyltransferase 1.
FDPS,"farnesyl diphosphate synthase (farnesyl pyrophosphate synthetase, dimethylallyltranstransferase, geranyltranstransferase).FPS.."
HPCAL1,hippocalcin-like 1.BDR1; HLP2; VILIP-3.hippocalcin-like 1.
LRP8,"low density lipoprotein receptor-related protein 8, apolipoprotein e receptor.APOER2; HSZ75190.low density lipoprotein receptor-related protein 8 isoform 3 precursor."
AP2S1,"adaptor-related protein complex 2, sigma 1 subunit.AP17; CLAPS2; AP17-DELTA.adaptor-related protein complex 2, sigma 1 subunit isoform AP17delta."
SQLE,squalene epoxidase..squalene monooxygenase.
HPCA,hippocalcin.BDR2.hippocalcin.
,"tubulin, beta, 2."
AP3S1,"adaptor-related protein complex 3, sigma 1 subunit.CLAPS3; Sigma3A.adaptor-related protein complex 3, sigma 1 subunit."
AP1S2,"adaptor-related protein complex 1, sigma 2 subunit.DC22; SIGMA1B; MGC:1902.adaptor-related protein complex 1 sigma 2 subunit."
TUBA1,"tubulin, alpha 1 (testis specific).FLJ30169; H2-ALPHA.tubulin, alpha 1."
AP2A1,"adaptor-related protein complex 2, alpha 1 subunit.ADTAA; CLAPA1; AP2-ALPHA.adaptor-related protein complex 2, alpha 1 subunit isoform 2."
SEC24D,"SEC24 related gene family, member D (S. cerevisiae).KIAA0755.Sec24-related protein D."
                 
Guess the column names for the whole table. There are only 2 columns in the table. Give only 1 answer for each column.
                 """},
            ]
print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)
test = pipe(messages)[0]['generated_text']
print(test[-1]['content'])

*** Pipeline:


{"colnames" : ["name", "description"]}


In [5]:
import os
import json
from json import loads, JSONDecodeError
from tqdm import tqdm
import numpy as np
import pandas as pd
from npy_postprocess import canonical_header
from f1_llama import report_gen


In [13]:
template_context = """
Column Names are limited to the following:
name, description, team, type, age, location, year, city, rank, status, state, category,
weight, code, club, artist, result, position, country, notes, class, company, album, symbol,
address, duration, format, county, day, gender, industry, language, sex, product, jockey,
region, area, service, teamName, order, isbn, fileSize, grades, publisher, plays, origin,
elevation, affiliation, component, owner, genre,  manufacturer, brand, family, credit, depth,
classification, collection, species, command, nationality, currency, range, affiliate,
birthDate, ranking, capacity, birthPlace, person, creator, operator, religion, education,
requirement, director, sales, continent, organisation
Do not use any column names aside from these.
"""

single_context = template_context + """
Output must be in valid JSON like the following example {"column" : "col1"}. Give only 1 prediction. Do NOT add any explanation beyond the JSON.

Given the following column values in a relational table:"""

multi_context = template_context + """
Output must be in valid JSON like the following example {"colnames" : ["col1", "col2"]}

Given the following relational table:
"""

sysprompt = {"role": "system", "content": """
            You are a database expert who can make general predictions for missing column values in database tables, and the predicted column names are within the required candidate set. All output must be in valid JSON. Don't add explanation beyond the JSON.
            Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
            """}

trues = []
preds = []

true_path = "npy/trues/"
pred_path = "npy/preds/" 
if not os.path.exists(true_path):
    os.makedirs(true_path)
if not os.path.exists(pred_path):
    os.makedirs(pred_path)

for tabledir in tqdm(directory_indexs):
    filenames = os.listdir(tabledir)
    real_cols = []
    pred_cols = []
    mismatch = 0
    error_num = 0
    #file_cnt = 0
    for filename in filenames[0:file_num]: #moved file_num here
        fullpath = tabledir + '/' + filename
        with open(fullpath) as f:
            pipe = pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=max_new_tokens,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.6,
                top_p=0.9,
                top_k=40,
                repetition_penalty=1.1
            )

            linelist = f.readlines()
            colnames = linelist[0][:-1].split(',')
            real_cols += colnames
            #if column_type == "single" and len(colnames) != 1:
            #    continue
            #if column_type == "multi" and len(colnames) <= 1:
            #    continue
            if column_type == "multi":
                lines = ''.join(linelist[1:rows_num+1])#.replace(',',';')
                content = f"{multi_context} \n {lines} Guess the column names for the whole table. There are only {len(colnames)} columns in the table. It is possible for multiple columns to have the same name.\n"
                messages = [sysprompt, {"role": "user", "content": content},]
                try:
                    jslist = loads(pipe(messages)[0]['generated_text'][-1]['content'])
                    
                    if len(jslist["colnames"]) == len(colnames):
                        #real_cols += colnames #commented because I already have it above
                        items = jslist["colnames"]
                        for item in items:
                            pred_cols.append(canonical_header(item))
                    else:
                        mismatch += 1
                        pred_cols += ["???"] * len(colnames) #removed ignore_mismatch
                except JSONDecodeError as e:
                    # below two lines for debug
                    # print("json load failed:", filename, outputs[0]["generated_text"][len(prompt):])
                    # print("error:", str(e))
                    pred_cols += ["???"] * len(colnames)
                    error_num += 1
            elif column_type == "single":
                df = pd.read_csv(fullpath).astype(str)
                for col in df.columns:
                    content = f'{single_context} {', '.join(df[col][0:rows_num])}\nGuess the column name'
                    messages = [sysprompt, {"role": "user", "content": content},]
                    try:
                        cpred = loads(pipe(messages)[0]['generated_text'][-1]['content'])["column"]
                        pred_cols.append(canonical_header(cpred))
                    except JSONDecodeError as e:
                        # below two lines for debug
                        # print("json load failed:", filename, outputs[0]["generated_text"][len(prompt):])
                        # print("error:", str(e))
                        pred_cols.append("???")
                        error_num += 1
        #file_cnt += 1
        #if file_cnt == file_num:
        #    break

    print(len(pred_cols), len(real_cols), mismatch, error_num)
    with open(f'npy/trues/{column_type}_{tabledir}_true.npy', 'wb') as f:
        np.save(f, np.array(real_cols, dtype='<U14'))
    with open(f'npy/preds/{column_type}_{tabledir}_pred.npy', 'wb') as f:
        np.save(f, np.array(pred_cols, dtype='<U14'))
    trues += real_cols
    preds += pred_cols
                

 20%|██        | 1/5 [06:31<26:06, 391.68s/it]

246 246 0 0
238 238 0 0


 60%|██████    | 3/5 [19:49<13:17, 398.59s/it]

260 260 0 0


 80%|████████  | 4/5 [25:08<06:06, 367.00s/it]

225 225 0 0


100%|██████████| 5/5 [30:24<00:00, 364.92s/it]

230 230 0 0


In [14]:
results_path = "results/" 
if not os.path.exists(results_path):
    os.makedirs(results_path)
print(len(preds), len(trues))
overall, report = report_gen(preds,trues)
with open(f"results/{column_type}_overall_{model_size}.json","w") as f:
    json.dump(overall,f)
report.to_csv(f'results/{column_type}_report_{model_size}.csv', index=False)
print(f"Results are successfully written into results/{column_type}_overall_{model_size}.json and results/{column_type}_report_{model_size}.csv")
# """correct = 0
# total = 0
# for real,pred in zip(real_cols, pred_cols):
#     for r,p in zip(real,pred):
#         total += 1
#         if r == (p[0].lower() + p[1:]):
#             correct += 1
# print(f'Accuracy: {correct/total}')"""

1199 1199
Results are successfully written into results/multi_overall_70B.json and results/multi_report_70B.csv


/afs/cs.wisc.edu/u/s/d/sdam/Documents/DataCleaningProject/CS839/f1_llama.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_report= pd.concat([df_report, pd.DataFrame(report[t],index=[0])], ignore_index=True)
